# This project follows correct step-by-step methods. 

In [ ]:
!pip install mtcnn
!pip install opencv-python

In [ ]:
import cv2
import os
from mtcnn import MTCNN # Multi Task Cascaded Convolution Networks

# Paths
raw_path = "/kaggle/input/celebrity-image-classification-dataset2/data"
cropped_path = "/kaggle/working/celebrity-image-classification/data_cropped"

detector = MTCNN() # for detecting only the face in image by cascading

for celeb in os.listdir(raw_path): #  os.listdir(raw_path) will list all the files and folders inside raw_path
    celeb_raw_folder = os.path.join(raw_path, celeb) # this will create a proper path for each celeb folder ('data/celeb_folder')
    celeb_cropped_folder = os.path.join(cropped_path, celeb) # specifying a path (location) to save the cropped faces of each celeb
    os.makedirs(celeb_cropped_folder, exist_ok = True) # makedirs(celeb_cropped_folder) creates that folder for the path. exist_ok = True will not throw error if folder already exists 


    # Loop through each image
    for img_name in os.listdir(celeb_raw_folder): # lists all the files (images) in that celebrity's image folder.
        img_path = os.path.join(celeb_raw_folder, img_name) # selects the full path of each image of that celeb folder
        img = cv2.imread(img_path) # loads the image into memory as a NumPy array. If the file valid image(jpg, png), you'll get a matrix of pixel values (shape: height, width, for 3 RGB colors)
        #if the image is corrupted or in wrong format, then it will return None.
        
        if img is None: # sometimes datasets have broken text files, images or hidden system files. This check avoid errors by skipping invalid files and moves on.
            continue 
        
        # here, the color of the image is being changed. Color image has 3 channels: Blue, Green, Red and shape: height, width, 3. A grayscale image has 1 channel: intensity, and shape: height, width which means the grey color is visible by changing the intensity of the image instead of colors

        faces = detector.detect_faces(img) # Given image to detect only face image stays in the same color (BGR)
        
        if faces: # If at leaset one face detected then run the below condition. Also, the 'face' here, returns a dictionary which stores the bounding box (the image rectangle), confidence(how correct it is classified as a face), and keypoints(the landmarks of each part: e.g., nose, eyes)
            
            face = max(faces, key = lambda f: f['box'][2]*f['box'][3]) # Selects the face(if multiple face per image) with the largest area(likely the main celeb) using h*w. This will avoid the model to select background images which can distrupt the model's learning.
            x, y, w, h = face['box']  # Here, each tuple (x, y, w, h) means - x, y is the top left corner of the face rectangle(box) and w, h is the width and height of the rectangle.
            
            x, y = max(0, x), max(0, y) # Useful if image is near the edge and the coordinates are detected -vly. So, max(0, x) stores the larger value only for x. max(0, x) compares both 0 and x and - if x > 0, x is stored, else 0.
            
            face_img = img[y:y+h, x:x+w] # slices(crops) the image to only extract the face region. face_imag is now a NumPy array containing only that face. (check the Brazil-covered notebook for further details).
            
            # “Use the original image name plus a number to save each cropped face uniquely in the right folder.”
            save_path = os.path.join(celeb_cropped_folder, f"{img_name.split('.')[0]}.jpg") # img_name.split('.')[0] takes the file name without extension.


            cv2.imwrite(save_path, face_img) # writes the image into the image path
            

## Preprocessing for CNN

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# these datagen's parameters(rescale, rotation_range, etc) are used during the training and the size of the training data will increase because of them. This is given to training because the model has to be trained on the basis of different image patterns so to avoid overfitting.
datagen = ImageDataGenerator(
    rescale = 1./255, # Since we know that a normal RGB image has pixel values in the range of 0 to 255 for each color channel (R, G, B). rescale = 1./255 divides each pixel value of image by 255 for normalization. E.g., [255,0,0] --> [1.0,0.0,0.0]
    rotation_range = 20, # randomly rotates image by 20%
    width_shift_range = 0.1, # shifts the image(cropped one) 10% horizontally. 
    height_shift_range = 0.1, # shifts the image 10% vertically
    horizontal_flip = True, # generates a mirrored image. Because the image of that celeb should also be learned as a mirrored image
    validation_split = 0.2 # splits dataset into 80% training, and 20% validation. Validation data is not augmented(i.e., these all parameters are not applied; except rescale)
)

train_gen = datagen.flow_from_directory(
    '/kaggle/input/cropped-dataset-of-celebrity-image/data_cropped',              # Path to the dataset folder
    target_size = (224, 224),    # Resizes the image(cropped one) into equal size. This size of the image is most commmon for this kind of problems.
    batch_size = 32,             # Number of images per batch
    class_mode = 'sparse',  # For giving labels to each folder image for multi-classification (one-hot labels). E.g., Akshay - label 1
    subset = 'training',          # Mentioned 'training' to use the training split (not the validation)
    shuffle = True            # Important for generalization
)

validation_gen = datagen.flow_from_directory(
    '/kaggle/input/cropped-dataset-of-celebrity-image/data_cropped',           # we need to specify it so that train and validation images are not overlapped
    target_size = (224, 224),
    batch_size = 32, # Number of images per batch
    class_mode = 'sparse',   # So that the model specifies the validation is correct/incorrect based on the same labels as of training
    subset = 'validation',
    shuffle = False        # not required here
)

2025-10-01 07:54:06.787274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759305246.809843    3432 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759305246.816867    3432 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 545 images belonging to 8 classes.
Found 130 images belonging to 8 classes.


In [ ]:
train_gen

### Defining the CNN model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

num_classes = len(train_gen.class_indices)

model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape = (224,224,3))) # 32 is 32 no. of filters that will be used for the initial layers. Each filter will be capturing different parts of the image and will give a feature map. Note, the early layers will use all the 32 filters simultaneously for a particular image at a single iteration.     # 3,3 is the filter size. activation function relu. 
model.add(MaxPooling2D(2,2)) # pooling layer of shape 2,2

model.add(Conv2D(64, (3,3), activation = 'relu')) # Now, using 64 filters for the remaining layers to extract more complex things like(shape of nose, eyes, arms) in middle layers.
model.add(MaxPooling2D(2,2))


model.add(Conv2D(128, (3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))

Flatten() # flattens the pooled data


model.add(Dense(128, activation = 'relu')) # A dense layer which will form a neural network by taking the flattened data to form a neral network of 128 neurons. Conv2D has filters whereas, Dense has neurons
model.add(Dropout(0.5)) # Randomly sets 50% of the neurons to 0 during training to prevent overfitting.

model.add(Dense(num_classes, activation = 'softmax')) # Output layer. Here we used no. of neurons = num_classes because, the final prediction should based on the final classes. Predicts proba for each class. softmax predicts the output class. softmax(make sure that the sum of all probas is 1)



model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy']) # accuracy tells keras to give the accuracy for validation and training at each epoch.


### Train model

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import os


# Compute class weights
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_gen.classes),# giving this to capture distinct classes in the data.
    y=train_gen.classes # Now, '.classes' will compute the class weights for the distinct classes instead of computing weights for all the records of train_gen.
)


class_weights_dict = dict(enumerate(class_weights_array))


history = model.fit(
    train_gen,
    validation_data = validation_gen,
    epochs = 55,
    class_weight=class_weights_dict
)

<h1>Hyperparameter Tuning using Bayesian Optimization</h1>

In [ ]:
# To decide how many trials are needed follow these steps:

#  Since the rule of thumb to decide the no. of trials which are required to find best model implies --> no. of trials = no. of parameters which are being tunned x 10.
# Now, the question comes if the range of the min-max value of the parameter is medium-large, than how many trials are required:
                # ✅ Small ranges (e.g. dropout 0.2–0.5, filters 32–128) → stick to n × 10.
                # ✅ Medium ranges (filters 32–512, LR 1e-5–1e-2) → go n × 12
                # ✅ Very wide ranges (LR 1e-7–1, dense 32–2048) → plan for 100+ trials or reduce the ranges.

# If you don't want to try the above steps then we should 1st) reduce the image size(target_size), 2nd) reduce the no. of batch size so that the model can become faster which will prevent less meemory overload and less GPU load because, when we take more batch size in CNN (not ANN) then, the further calculations of filters kernels and padding becomes much more time consuming. So, taking less batch_size makes the model more faster and accurate and prevent this problem.


In [2]:
train_gen_small = datagen.flow_from_directory(
    '/kaggle/input/cropped-dataset-of-celebrity-image/data_cropped',               
    target_size = (128, 128),  # smaller target and batch size for fast tuning   
    batch_size = 8,              
    class_mode = 'sparse',   
    subset = 'training',
    shuffle = True
    
)

validation_gen_small = datagen.flow_from_directory(
    '/kaggle/input/cropped-dataset-of-celebrity-image/data_cropped',            
    target_size = (128, 128),
    batch_size = 8, 
    class_mode = 'sparse',    
    subset = 'validation',
    shuffle = False
    
)

Found 545 images belonging to 8 classes.
Found 130 images belonging to 8 classes.


In [ ]:
pip install keras-tuner

In [ ]:
import keras_tuner as kt
import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import os

num_classes = len(train_gen_small.class_indices)


def model_builder(hp):
    model2 = Sequential()
    
    model2.add(Conv2D(
        filters = 64,
        kernel_size = (3,3), # tuning filter size where the possible sizes to tune are 3*3 or 5*5
        activation = "relu",
        padding = "same", # This adds 0 to the extra size to make the ouput of the equal size of the input size
        input_shape = (224, 224, 3) # Using the same size of the resized(cropped) size of the images

    ))
    model2.add(MaxPooling2D(pool_size = 2)) # 2 x 2 pooling shape of the max pooling


    # Second Conv Layer
    model2.add(Conv2D(
        filters = 128,
        kernel_size = (3,3),
        activation = "relu",
        padding = "same"
        
    ))
    model2.add(MaxPooling2D(pool_size = 2)) # 2 x 2 pooling shape of the max pooling


    # Third Conv Layer
    model2.add(Conv2D(
        filters = 256,
        kernel_size = (3,3),
        activation ="relu",
        padding = "same"
    
    ))
    model2.add(MaxPooling2D(pool_size = 2)) # 2 x 2 pooling shape of the max pooling
    
    model2.add(Flatten())
    
    # Tuning Dnese Layer
    model2.add(Dense(
        units = hp.Int("dense_units", 64, 256, step = 64), # Tuning no. of neurons
        activation = "relu"
        
    ))
    

    # Tuning Dropout
    model2.add(Dropout(
        hp.Float("dropout", 0.2, 0.5, step = 0.1)
        
    ))

    
    # Defining Output Layer
    model2.add(Dense(num_classes, activation = "softmax"))

    # Compiling model
    model2.compile(
        optimizer = keras.optimizers.Adam(
            hp.Float("lr", 1e-5, 1e-2, sampling = "log")),
            loss = "sparse_categorical_crossentropy",
            metrics = ["accuracy"]
    )

    return model2


# Tuner

tuner = kt.BayesianOptimization(
    model_builder,
    objective = "val_accuracy",
    max_trials = 26,
    directory = "bo_tuner",
    project_name = "cnn_fixed_layers"
    
)

# EarlyStopping
es = EarlyStopping(
    monitor = "val_loss",
    patience = 5,
    restore_best_weights = True
)

# Compute class weights
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_gen_small.classes),# giving this to capture distinct classes in the data.
    y=train_gen_small.classes # Now, '.classes' will compute the class weights for the distinct classes instead of computing weights for all the records of train_gen.
)


class_weights_dict = dict(enumerate(class_weights_array))


# Runinig Search
tuner.search(train_gen_small,
            validation_data = validation_gen_small,
            epochs = 20,
            class_weight = class_weights_dict,
            callbacks = [es]
            
            )

In [ ]:
sample_weights = list(class_weights_dict.values)
sample_weights

# Evaluate

In [ ]:
print(keras.__version__)

In [ ]:
loss, acc = model.evaluate(validation_gen)
print("Validation Accuracy: ", acc)